# Librerias

In [3]:
import os
import pyodbc
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Conexión SSMS y Consulta Vision Cliente

**Vision_Cliente**

En el siguiente  código se realiza una conexión con el servidor local (SSMS). A través de esta conexión, se ejecuta una consulta SQL almacenada en el archivo `vision_cliente.sql`, ubicado en la carpeta `queries`.

Esta consulta, explicada con más detalle en el archivo de apoyo `apoyo.md`, permite obtener una tabla final con información por cliente. En concreto, se calculan una serie de métricas agregadas relacionadas con el comportamiento de compra, mantenimiento, rentabilidad del cliente.

Además, se incluyen datos identificativos  de cada cliente (como la edad, el género o el status social).

In [4]:
# Conexión SSMS
LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'dwh_case1'
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'
local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

# Conexión a la base de datos local
conn_local = pyodbc.connect(local_conn_str)
cursor_local = conn_local.cursor()

# Ruta archivo vision_cliente.sql
queries_notebooks_path = os.path.join(os.getcwd(), "queries")
vision_cliente_file = os.path.join("..", "queries", "vision_cliente.sql")


# 1. Leer la consulta desde vision_cliente.sql
with open(vision_cliente_file, 'r', encoding='utf-8') as file:
    vision_query = file.read()

# 2. Ejecutar la consulta en SQL Server Local y obtener un DataFrame
print("Ejecutando la consulta vision_cliente.sql ")
df = pd.read_sql(vision_query, conn_local)

# Guardar los resultados de la consulta en un archivo CSV
df.to_csv("vision_cliente.csv", index=False, encoding='utf-8')
print("Archivo 'vision_cliente.csv' guardado correctamente.")


print("Columnas del DataFrame 'vision_cliente':")
for col in df.columns:
    print(f"- {col}")

print(f"Datos extraídos: {df.shape[0]} filas.")

# 3. Convertir NaN a 0 y ajustar tipos
df = df.fillna(0)

for col in df.select_dtypes(include=['float64']).columns:
    df[col] = df[col].astype(np.float32)

for col in df.select_dtypes(include=['int64']).columns:
    df[col] = df[col].astype(np.int32)

# 4. Crear la tabla 'vision_cliente' (borrar si ya existe)
drop_table_sql = "DROP TABLE IF EXISTS vision_cliente"
cursor_local.execute(drop_table_sql)
conn_local.commit()

# 5. Generar CREATE TABLE dinámico en base a los tipos del DataFrame
create_table_sql = f"""
CREATE TABLE vision_cliente (
    {', '.join([
        f'[{col}] FLOAT' if df[col].dtype == np.float32 
        else f'[{col}] INT' if df[col].dtype == np.int32 
        else f'[{col}] NVARCHAR(255)' for col in df.columns
    ])}
);
"""
cursor_local.execute(create_table_sql)
conn_local.commit()

print("Tabla 'vision_cliente' creada correctamente.")

# 6. Insertar los datos
placeholders = ', '.join(['?' for _ in df.columns])
insert_sql = f"INSERT INTO vision_cliente VALUES ({placeholders})"

cursor_local.fast_executemany = True
cursor_local.executemany(insert_sql, df.values.tolist())
conn_local.commit()

print(f"{df.shape[0]} filas insertadas en 'vision_cliente'.")

# Cerrar conexiones
conn_local.close()

print("\n¡Consulta Cliente Terminada!")


Ejecutando la consulta vision_cliente.sql 
Archivo 'vision_cliente.csv' guardado correctamente.
Columnas del DataFrame 'vision_cliente':
- Customer_ID
- Edad
- RENTA_MEDIA_ESTIMADA
- GENERO
- Fecha_nacimiento
- STATUS_SOCIAL
- CODIGO_POSTAL
- provincia
- poblacion
- lat
- lon
- Origen_Compra
- Fue_Lead_Alguna_Vez
- Numero_Veces_Lead
- Total_Compras
- PVP_Total
- PVP_Medio
- PVP_Diferencia
- Dias_Entre_Primera_Ultima_Compra
- Dias_Desde_Ultima_Compra
- Compra_Finde_O_Festivo
- Edad_Media_Coche
- Total_Revisiones
- Km_Medio_por_Revision
- Tuvo_Mantenimiento_Gratuito
- Dias_Medio_En_Taller
- Total_Quejas
- Compra_Tienda_Unica
- Contrato_Seguro_Bateria
- Coste_Medio_Cliente
- Margen_Bruto_Medio
- Margen_eur_Medio
- Rentabilidad_Relativa
- Churn_Cliente
- Dias_Medios_Desde_Ultima_Revision
Datos extraídos: 44053 filas.
Tabla 'vision_cliente' creada correctamente.
44053 filas insertadas en 'vision_cliente'.

¡Consulta Cliente Terminada!
